## RB Benchmarks on the IBMQ ##

In [ ]:
import random
import math
import numpy as np

import copy


import sys, os, time

from qinfer import LiuWestResampler
from qinfer import utils

from qiskit import IBMQ
from qiskit import QuantumCircuit, execute, Aer
import qiskit.ignis.verification.randomized_benchmarking as rb

IBMQ.load_account()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sbs

import smc
Distribution = smc.Distribution

sbs.set(style="darkgrid")

%matplotlib inline

Invert and measure circuit adder

In [ ]:
def design_circuit(n_qubits, inv_arr, circuit=None):
    
    if circuit is None:
        circuit = QuantumCircuit(n_qubits, n_qubits)
    
    for i, element in enumerate(inv_arr):
        if element == 1:
            circuit.x(i)
    
    circuit.measure(list(range(n_qubits)), list(range(n_qubits)))
    return circuit

Random circuit builder

In [ ]:
def random_circuit(n_qubits):
    length_vector = [1]
    n_seeds = 1
    
    initial_list = list(range(0, n_qubits))    
    rb_pattern = []
    
    while len(initial_list) > 1:
        sample = random.sample(initial_list, 2)
        [initial_list.remove(i) for i in sample]
        rb_pattern.append(sample)
    
    if len(initial_list) == 1:
        rb_pattern.append([initial_list[0]])
    
    
    circuit, xdata = rb.randomized_benchmarking_seq(
        length_vector = [1],
        nseeds = 1,
        rb_pattern = rb_pattern
        )
    circuit = circuit[0][0]
    cregs = circuit.cregs
    
    circuit.remove_final_measurements(inplace=True)
    circuit.cregs = cregs
    
    for i in range(n_qubits):
        circuit.barrier(i)
        if (round(random.random())):
            circuit.x(i)
            circuit.barrier(i)
    
    return circuit

Some temporary and now unused code to check everything is working without hitting the IBMQ Backend

In [ ]:
def fake_experiment(true_bias, inversion_arr):
    result = []
    for bias, i in zip(true_bias, inversion_arr):
        outcome = i if random.random() < bias[i] else 1 - i
        result.append(outcome)
    return result

Running RB benchmarks using SMB

In [ ]:
provider = IBMQ.get_provider(group='open', project='main')
backend = provider.get_backend('ibmq_vigo')

n_qubits = 5

n_measurements = 200
n_experiments = 10
n_points = 4000
n_qubits = 5

results = []

for i in range(n_experiments):    

    result_data = {}
    
    circuit = random_circuit(n_qubits)

    result_data['circuit'] = circuit
    result_data['risk'] = []
    result_data['mean'] = []
    result_data['len'] = len(circuit)
    
    dist = Distribution(n_points=n_points, n_qubits=n_qubits)


    for _ in range(n_measurements):
        
        inversion_arr = dist.next_experiment()       
        
        tmp_circuit = copy.deepcopy(circuit)
        tmp_circuit = design_circuit(n_qubits, inversion_arr, circuit=tmp_circuit)

        job = execute(tmp_circuit, backend, shots=1)
        
        result = job.result()
        outcome = list(map(int, list(list(result.get_counts(circuit).keys())[0])))     
        
        print("Measurements: {} Outcome: {}".format(inversion_arr, outcome))
        dist.measure(outcome, inversion_arr)
        
        result_data['risk'].append(dist.calc_bayes_risk())
        result_data['mean'].append(dist.calc_bayes_mean())

    
    result_data['distrubtion'] = dist
    results.append(result_data)


More RB benchmarks

In [ ]:
n_points = 4000
n_qubits = 5

vals = []
m_outcomes = {}
m_outcomes_order = []

dist = Distribution(n_points=n_points, n_qubits=n_qubits)

for m_number, (measurement, outcome) in enumerate(zip(measurements_arr, outcomes_arr)):    

    dist.measure(outcome, measurement)

    vals.append([dist.calc_bayes_mean(), dist.calc_bayes_risk()])
    
    bayes_mean = dist.calc_bayes_mean()
    m_arr = [1 if bayes_mean[2 * i + 1] < bayes_mean[2 * i] else 0 for i in range(len(bayes_mean) // 2)]
           
    m_arr_str = ''.join(list(map(str, m_arr)))
        
    print(m_arr_str)
    
    if m_number % 10 == 0:
        print("Calculating outcome: ", m_number)
        m_outcomes_order.append(m_arr_str)
        
        if m_arr_str not in m_outcomes:
        
            job = execute(tmp_circuit, backend, shots=4000)

            result = job.result()

            m_outcomes[m_arr_str] = result.get_counts(circuit)    


In [ ]:
ex_res = []
for i in range(0, len(measurements_arr), 15):
    
    inversion_arr = measurements_arr[i]       
        
    tmp_circuit = copy.deepcopy(circuit)
    tmp_circuit = design_circuit(n_qubits, inversion_arr, circuit=tmp_circuit)
    
    job = execute(tmp_circuit, backend, shots=1000)
        
    result = job.result()
    outcome = result.get_counts(circuit)
    
    ex_res.append(outcome)
    print(inversion_arr)
    

In [ ]:
def bootstrap(result, target=0, samples=900, repeats=1000):
    sampler = []
    for i in result:
        for _ in range(result[i]):
            sampler.append(int(i))
            
    probs = []
    
    for _ in range(repeats):
        tally = 0
        for i in range(samples):
            sample = sampler[random.randint(0, len(sampler) - 1)]
            
            if int(target) == sample:
                tally += 1
        
        probs.append(tally / samples)
    return probs

In [ ]:
n_repeats = 100
x_coords = np.array([[i * 10] * n_repeats for i in range(len(m_outcomes_order))]).flatten()
y_coords = 1 - np.array(
    [bootstrap(
        m_outcomes[m_outcomes_order[i]], 
        target = max(m_outcomes[m_outcomes_order[i]], key=lambda key: m_outcomes[m_outcomes_order[i]][key]), 
        samples=900, repeats=n_repeats) 
     for i in range(len(m_outcomes_order))
    ]).flatten()


Make plots

In [ ]:
x_coords_adj = [min(i, 1 - i) for i in x_coords]

ax = sbs.pointplot(x_coords, y_coords, ci=100)

ax.lines[0].set_marker('^')
ax.lines[0].set_linestyle('--')

# london = mlines.Line2D([], [], color='blue', marker='.',
#                           markersize=10, label='IBMQ Virgo')

plt.xlabel("Characterisation Measurements")
plt.ylabel("Error Probability")
plt.title("Randomised Depth 14 Circuit Error Rates (IBMQ Melbourne)")
plt.savefig('figs/error_rates_rb_melbourne.pdf')